# Apache Spark Koalas

Este notebook contiene las funciones principales de Koalas, obtenidas de la documentación oficial de https://koalas.readthedocs.io/

In [ ]:
import sys
sys.executable

In [ ]:
pip install findspark

In [ ]:
import findspark
findspark.init()

import pandas as pd
import pyspark

import pandas as pd
import numpy as np
import databricks.koalas as ks
from pyspark.sql import SparkSession

### 1. Creación de objetos



Creando una serie Koalas pasando una lista de valores, permitiendo que Koalas cree un índice entero predeterminado:

In [ ]:
s = ks.Series([1, 3, 5, np.nan, 6, 8])

In [ ]:
s

Creando un Koalas DataFrame pasando un dict de objetos que se pueden convertir a series.

In [ ]:
kdf = ks.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

In [ ]:
kdf

Creando un DataFrame de pandas pasando una matriz numpy, con un índice de fecha y hora y columnas etiquetadas:

In [ ]:
dates = pd.date_range('20130101', periods=6)

In [ ]:
dates

In [ ]:
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))

In [ ]:
pdf

Ahora, este DataFrame de pandas se puede convertir en un DataFrame de Koalas

In [ ]:
kdf = ks.from_pandas(pdf)

In [ ]:
type(kdf)

Sin embargo, se ve y se comporta igual que un DataFrame de pandas

In [ ]:
kdf

Además, es posible crear un **Koalas DataFrame desde Spark DataFrame**.

Creando un Spark DataFrame a partir de pandas DataFrame

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
sdf = spark.createDataFrame(pdf)

In [ ]:
sdf.show()

Creando Koalas DataFrame desde Spark DataFrame.
`to_koalas ()` se adjunta automáticamente a Spark DataFrame y está disponible como una API cuando se importa Koalas.

In [ ]:
kdf = sdf.to_koalas()

In [ ]:
kdf

Tiene [dtypes] específicos. Actualmente se admiten los tipos que son comunes a Spark y pandas.

In [ ]:
kdf.dtypes

### 2. Manipulación de datos


A diferencia de los pandas, los datos en un dataframe de datos de Spark no están _ordenados_, no tienen una noción intrínseca de índice. Cuando se le solicite el encabezado, Spark solo tomará el número solicitado de filas de una partición. **No hay que utilizar el df de Koalas para devolver filas específicas**, use `.loc` o` iloc` en su lugar.

In [ ]:
kdf.head()

Muestre el índice, las columnas y los datos numéricos subyacentes.

También puede recuperar el índice; la columna de índice se puede atribuir a un DataFrame, ver más adelante

In [ ]:
kdf.index

In [ ]:
kdf.columns

In [ ]:
kdf.to_numpy()

**Describe** muestra un resumen estadístico rápido de sus datos

In [ ]:
kdf.describe()

Transposición de sus datos

In [ ]:
kdf.T

Ordenando por su índice

In [ ]:
kdf.sort_index(ascending=False)

Ordenar por valor

In [ ]:
kdf.sort_values(by='B')

### 3. Datos faltantes
Koalas utiliza principalmente el valor `np.nan` para representar los datos faltantes. Por defecto, no se incluye en los cálculos.


In [ ]:
pdf1 = pdf.reindex(index=dates[0:4], columns=list(pdf.columns) + ['E'])

In [ ]:
pdf1.loc[dates[0]:dates[1], 'E'] = 1

In [ ]:
kdf1 = ks.from_pandas(pdf1)

In [ ]:
kdf1

Para eliminar las filas que tienen datos faltantes.

In [ ]:
kdf1.dropna(how='any')

Llenando los datos faltantes.

In [ ]:
kdf1.fillna(value=5)

### 4. Operaciones

#### Estadísticas
Las operaciones en general excluyen los datos faltantes.

Realización de una estadística descriptiva:

In [ ]:
kdf.mean()

#### Configuraciones de Spark

Varias configuraciones en PySpark se pueden aplicar internamente en Koalas.
Por ejemplo, puede habilitar la optimización de Arrow para acelerar enormemente la conversión de pandas internos.

In [ ]:
prev = spark.conf.get("spark.sql.execution.arrow.enabled")  # Keep its default value.
ks.set_option("compute.default_index_type", "distributed")  # Use default index prevent overhead.

import warnings
warnings.filterwarnings("ignore")  # Ignore warnings coming from Arrow optimizations.

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", True)
%timeit ks.range(300000).to_pandas()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", False)
%timeit ks.range(300000).to_pandas()

In [ ]:
ks.reset_option("compute.default_index_type")
spark.conf.set("spark.sql.execution.arrow.enabled", prev)  # Set its default value back.

#### Agrupación
Por "agrupar por" nos referimos a un proceso que involucra uno o más de los siguientes pasos:

- Dividir los datos en grupos según algunos criterios.
- Aplicar una función a cada grupo de forma independiente
- Combinar los resultados en una estructura de datos

In [ ]:
kdf = ks.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B': ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                    'C': np.random.randn(8),
                    'D': np.random.randn(8)})

In [ ]:
kdf

Agrupar y luego aplicar el **sum** a los grupos resultantes.

In [ ]:
kdf.groupby('A').sum()

In [ ]:
kdf.groupby(['A', 'B']).sum()

### 5. Visualización de datos: Generar gráficos


In [ ]:
pser = pd.Series(np.random.randn(1000),
                 index=pd.date_range('1/1/2000', periods=1000))

In [ ]:
kser = ks.Series(pser)

In [ ]:
kser = kser.cummax()
kser

**Gráficos de matplotlib**

In [ ]:
ks.options.plotting.backend = "matplotlib"

kser.plot()

**Gráficos de Pandas Bokeh**

In [ ]:
ks.options.plotting.backend = "pandas_bokeh"

kdf.plot(backend="pandas_bokeh", title="Example Figure")
kser.plot()


**Gráfico por defecto con plotly**

In [ ]:
kser.plot()

En un DataFrame, el plot() es una conveniencia para trazar todas las columnas con etiquetas:

In [ ]:
pdf = pd.DataFrame(np.random.randn(1000, 4), index=pser.index,
                   columns=['A', 'B', 'C', 'D'])

In [ ]:
kdf = ks.from_pandas(pdf)

In [ ]:
kdf = kdf.cummax()
kdf

In [ ]:
kdf.plot()

### 6. Entrada / salida de datos


### CSV

CSV es sencillo y fácil de usar

In [ ]:
kdf.to_csv('foo.csv')
ks.read_csv('foo.csv').head(10)

### Parquet

Parquet es un formato de archivo eficiente y compacto para leer y escribir más rápido.

In [ ]:
kdf.to_parquet('bar.parquet')
ks.read_parquet('bar.parquet').head(10)

### Spark IO

Además, Koalas es totalmente compatible con las diversas fuentes de datos de Spark, como ORC y una fuente de datos externa.

In [ ]:
kdf.to_spark_io('zoo.orc', format="orc")
ks.read_spark_io('zoo.orc', format="orc").head(10)